# Quantum Computing Basics

**Perquisites:** Linear Algebra, Quantum Mechanics and Python

In [10]:
import tensorcircuit as tc
import numpy as np

## Qubits

**Superposition**: In classical computation, the unit of information is a bit, which can be 0 or 1. In quantum computation, this unit is a quantum bit (qubit), which is a superposition of 0 and 1 ($|0\rangle$ and $|1\rangle$ in Dirac notation). We identify the two basis states with the vectors $\binom{1}{0}$ and $\binom{0}{1}$, respectively. A single qubit can be in any superposition

$$
\alpha_0|0\rangle+\alpha_1|1\rangle=\binom{\alpha_0}{\alpha_1},
$$

with $\left|\alpha_0\right|^2+\left|\alpha_1\right|^2=1.$

**Multiple-qubit system**: Similarly we can think of systems of more than 1 qubit, which "live" in the tensor product space of multiple-qubit systems. For instance, a 2-qubit system has 4 basis states: $|0\rangle \otimes|0\rangle,|0\rangle \otimes|1\rangle,|1\rangle \otimes|0\rangle,|1\rangle \otimes|1\rangle$. Here for instance $|1\rangle \otimes|0\rangle$ means that the first qubit is in its basis state $|1\rangle$ and the second qubit is in its basis state $|0\rangle$. We will often abbreviate this to $|1\rangle|0\rangle$,or $|10\rangle $, which is equivalent 

$$
|10\rangle = \binom{0}{1}\otimes\binom{1}{0} = \begin{pmatrix} 0\\ 0\\ 1\\ 0\end{pmatrix}
$$

in vector space. 

More generally, a register of $n$ qubits has $N=2^n$ basis states, each of the form $\left|b_1 b_2 \cdots b_n\right\rangle$, with $b_i \in\{0,1\}$. We often abbreviate $0 \cdots 0$ to $0^n$. Since bitstrings of length $n$ can be viewed as numbers between 0 and $2^n-1$, we can also write the basis states as numbers $|0\rangle,|1\rangle,|2\rangle, \cdots,\left|2^n-1\right\rangle$. A quantum state of $n$ qubits can be in any superposition

$$
\vert \psi\rangle=\sum_{j=0}^{2^n-1}\alpha_j|j\rangle
$$

with $ \sum_{j=0}^{2^n-1}\left|\alpha_j\right|^2=1$. The adjoint bra state for the ket state is defined as  $\langle \psi \vert = \sum_{j=0}^{2^n-1}\alpha_j^*\langle j\vert$.


If we measure $\vert \psi\rangle$ in the computational basis, we obtain the $n$-bit state state $|j\rangle$ with probability $\left|\alpha_j\right|^2$

**Overlap**: The overlap between two quantum states in  $n$-qubit space ($N=2^n$) is expressed as follows in Dirac notation and explicit vectors:

$$
\langle \psi\vert \phi\rangle = \begin{pmatrix}\psi_0^*, \psi_1^*, \cdots \psi_{N-1}^*\end{pmatrix}\begin{pmatrix} \phi_0\\ \phi_1 \\ \cdots\\ \phi_{N-1}\end{pmatrix}=\sum_{i=0}^{2^n-1} \psi_i^*\phi_i.
$$

*Excercise:* Quantum basis state $\vert 5\rangle$ for 4-qubit system.

In [14]:
s = f"{5:04b}"
print(f"basis state |{s}>")
v1 = np.array([0, 1.0])
v0 = np.array([1, 0.0])
v_list = [v0, v1]


def tensor_product(*vs):
    if len(vs) > 2:
        return np.kron(vs[0], tensor_product(*vs[1:]))
    elif len(vs) == 2:
        return np.kron(vs[0], vs[1])
    else:
        raise ValueError("The number of vectors is not allowed")


print(f"the state in vector form is {tensor_product(*[v_list[int(i)] for i in s])}")

# the nonzero element is exact the 5-th (0-indexed)

basis state |0101>
the state in vector form is [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## Quantum gates

A unitary that acts on several qubits is called a quantum gate, in analogy to classical logic gates like AND, OR, NOT.


**Single-qubit gates**:

Some typical single-qubit gates (2*2 unitary matrix).

1. $X$: the bitflip gate which negates the bit, i.e., $X\vert 0\rangle = \vert 1\rangle, X\vert 1\rangle = \vert 0 \rangle$,
$$
X=\begin{pmatrix}
0 & 1\\
1&0
\end{pmatrix}
.
$$
2. $Z$: the phaseflip gate which puts a $−$ in front of $\vert 1\rangle,
$$
Z=\begin{pmatrix}
1 & 0\\
0& -1
\end{pmatrix}
.
$$
3. $Y$: $Y=iXZ$ which applies both bitflip and phaseflip on the qubit,
$$
Y=\begin{pmatrix}
0 & -i\\
i& 0
\end{pmatrix}
.
$$

4. $H$: Hadamard gate, which is a single-qubit analogy for Fourier transformation, i.e. $H\vert 0\rangle = \frac{1}{\sqrt{2}}(\vert 0\rangle +\vert 1\rangle)$, $H\vert 1\rangle = \frac{1}{\sqrt{2}}(\vert 0\rangle -\vert 1\rangle)$, with the matrix form
$$
H=\frac{1}{\sqrt{2}}\begin{pmatrix}
1 & 1\\
1& -1
\end{pmatrix}
.
$$ 


5. $R_\phi $: single qubit phase rotation gate, i.e. $R_\phi \vert 0\rangle = \vert 0\rangle, R_\phi\vert 1\rangle = e^{i\phi}\vert 1\rangle$,
$$
R_\phi=\begin{pmatrix}
1 & 0\\
0& e^{i\phi}
\end{pmatrix}
.
$$ 

Note that $Z=R_\pi$ and $T=R_{\pi/4}$.

Also note that both quantum gates and quantum states have an overall phase redundency with no physical information. Namely, $\vert 00\rangle + \vert 11\rangle$ and $-\vert 00\rangle -\vert 11\rangle$ are the same states, and $R_\phi$ gate can also be expressed as:
$$
R_\phi=\begin{pmatrix}
e^{-i\phi/2} & 0\\
0& e^{i\phi/2}
\end{pmatrix}
.
$$ 

**Inverse gate**:

If $UU^{-1}=I$, we call the unitary $U^{-1}$ the inverse gate of $U$, e.g. $X^{-1}=X$, $H^{-1}=H$, $R_\phi^{-1}=R_{-\phi}$.

**Two-qubit gates**:


## Quantum Measurements

## Quantum Circuits

## Quantum complexity theory